In [1]:
from openpyxl import load_workbook
from openpyxl import workbook
import os
import win32com.client as win32
import pandas as pd
from unidecode import unidecode

In [2]:
## COVERTER DE XLS PARA XLSX
open_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\xls\dados_industria.xls'
save_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\xlsx\dados_industria.xls'

xlsx_format = save_filepath+"x"

#Verifica se já existe um arquivo e remove
if os.path.exists(xlsx_format) == True:
    os.remove(xlsx_format)
    
excel = win32.Dispatch('Excel.Application')
#excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(open_filepath)

wb.SaveAs(save_filepath+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
wb.Close()                                       #FileFormat = 56 is for .xls
excel.Application.Quit()

In [3]:
##CARREGANDO PLANILHA
wb = load_workbook(xlsx_format)
sheets = wb.sheetnames
print(sheets)

#SELECIONAR UMA WORKSHEETS (inicia em 0)
sheets = sheets[0] # <- de 0 até 15

ws = wb[sheets]
print(ws, "Ativo")

['PRODUÇÃO', 'EMPREGADOS', 'UCI (%)', 'UCI (efetiva-usual)', 'ESTOQUES (evolução)', 'ESTOQUES (efetivo-planejado)', 'EXPECTATIVAS - DEMANDA', 'EXPECTATIVA - EXPORTAÇÃO', 'EXPECTATIVA - COMPRAS', 'EXPECTATIVA - EMPREGADOS', 'EXPECTATIVA - INVESTIMENTO', 'SITUACAO FINANCEIRA LUCRO', 'SITUACAO FINANCEIRA PREÇO MEDIO', 'SITUACAO FINANCEIRA', 'SITUACAO FINANCEIRA CREDITO', 'PRINCIPAIS_PROBLEMAS']
<Worksheet "PRODUÇÃO"> Ativo


# COLETA DE DADOS

In [4]:
##LENDO OS MESES
    
max_linha = ws.max_row
max_coluna = ws.max_column

valores = []

for i in range(2, max_coluna + 1):
    valores.append(ws.cell(row=8, column=i).value)

df_dados = pd.DataFrame(valores,  columns =  ["Data"])
    
##CRIANDO DATA FRAME COM OS DADOS DE PRODUÇÃO

linhas = 11

while linhas <= 54:
    
    if linhas > 54:
        continue
        
    max_coluna = ws.max_column
    valores_r = []
    
    for i in range(linhas, linhas + 1):
        var = (ws.cell(row=i, column = 1).value)
    for j in range(2, max_coluna + 1): 
        valores_r.append(ws.cell(row = i, column=j).value)
    df_dados.insert(1, var, valores_r)
    
    linhas += 1

# LIMPANDO OS DADOS

In [5]:
#RETIRA AS COLUNAS VAZIAS
df_dados = df_dados.drop(columns = ["POR PORTE", "POR SETOR"])

#RETIRA LINHA VAZIAS
df_dados.dropna()

,Data,"Manutenção, reparação e instalação de máquinas e equipamentos.",Produtos diversos,Móveis,Outros equipamentos de transporte,"Veículos automotores, reboques e carrocerias",Máquinas e equipamentos,"Máquinas, aparelhos e materiais elétricos","Equipamentos de informática, produtos eletrônicos e outros",Produtos de metal (exceto máquinas e equipamentos),...,"Extração de carvão, petróleo e gás natural",IND. EXTRATIVA,GRANDE,MÉDIA,PEQUENA,CENTRO-OESTE,SUL,SUDESTE,NORDESTE,NORTE
0,jan10,-,50.0,42.7,-,53.6,51.6,50.0,50.6,50.0,...,-,48.5,51.3,46.5,46.5,48.6,48.9,48.1,49.0,53.7
1,fev10,-,56.9,47.1,-,52.7,57.6,50.0,58.3,48.6,...,-,48.4,52.3,50.0,48.0,52.0,55.3,49.4,46.1,49.3
2,mar10,-,62.5,63.3,-,67.5,64.8,65.2,65.7,61.0,...,-,58.1,65.2,62.8,56.3,61.5,64.0,64.3,59.4,60.9
3,abr10,-,51.0,48.6,-,51.8,57.0,50.9,54.6,50.0,...,-,53.5,51.7,51.6,47.6,51.8,52.5,49.7,48.7,53.7
4,mai10,-,57.3,50.5,-,53.5,55.7,48.1,55.4,53.7,...,-,59.7,57.2,54.3,49.5,54.9,53.7,54.9,55.7,57.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2020-02-01 00:00:00,46.4,43.8,42.9,45.8,48.3,47.8,46.0,44.7,47.1,...,-,46.3,50.9,45.2,43.2,48.2,51.7,46.2,46.0,43.2
122,2020-03-01 00:00:00,34.6,30.3,14.2,28.8,30.2,31.0,36.1,31.1,26.7,...,-,45.6,35.0,35.1,28.0,34.4,30.4,32.4,30.3,35.2
123,2020-04-01 00:00:00,28.7,17.6,9.7,23.4,16.8,26.1,22.0,22.9,22.3,...,-,43.0,26.6,26.9,24.0,39.0,22.8,25.7,23.2,27.7
124,2020-05-01 00:00:00,44.3,38.7,40.2,42.6,49.2,45.5,36.1,34.8,40.0,...,-,48.4,47.2,42.4,35.6,50.0,45.3,41.9,41.2,39.9


# CORRIGINDO O CAMPO DATA

In [6]:
df_data = df_dados["Data"]


df_data = df_data.replace({'fev': 'feb'}, regex=True)
df_data = df_data.replace({'abr': 'apr'}, regex=True)
df_data = df_data.replace({'mai': 'may'}, regex=True)
df_data = df_data.replace({'ago': 'aug'}, regex=True)
df_data = df_data.replace({'set': 'sep'}, regex=True)
df_data = df_data.replace({'out': 'oct'}, regex=True)
df_data = df_data.replace({'dez': 'dec'}, regex=True)

df_data = pd.to_datetime(df_data, format='%b%y', errors="coerce").fillna(pd.to_datetime(df_data, format="%Y-%m-%d %H:%M:%S", errors="coerce"))

df_data = pd.DataFrame(df_data)

In [7]:
df_data.head(76)

,Data
0,2010-01-01
1,2010-02-01
2,2010-03-01
3,2010-04-01
4,2010-05-01
...,...
71,2015-12-01
72,2016-01-01
73,2016-02-01
74,2016-03-01


In [8]:
df_dados = df_dados.drop(columns="Data")

In [9]:
df_dados = pd.merge(df_dados, df_data, left_index=True, right_index=True)
df_dados

,"Manutenção, reparação e instalação de máquinas e equipamentos.",Produtos diversos,Móveis,Outros equipamentos de transporte,"Veículos automotores, reboques e carrocerias",Máquinas e equipamentos,"Máquinas, aparelhos e materiais elétricos","Equipamentos de informática, produtos eletrônicos e outros",Produtos de metal (exceto máquinas e equipamentos),Metalurgia,...,IND. EXTRATIVA,GRANDE,MÉDIA,PEQUENA,CENTRO-OESTE,SUL,SUDESTE,NORDESTE,NORTE,Data
0,-,50.0,42.7,-,53.6,51.6,50.0,50.6,50.0,51.1,...,48.5,51.3,46.5,46.5,48.6,48.9,48.1,49.0,53.7,2010-01-01
1,-,56.9,47.1,-,52.7,57.6,50.0,58.3,48.6,57.3,...,48.4,52.3,50.0,48.0,52.0,55.3,49.4,46.1,49.3,2010-02-01
2,-,62.5,63.3,-,67.5,64.8,65.2,65.7,61.0,65.7,...,58.1,65.2,62.8,56.3,61.5,64.0,64.3,59.4,60.9,2010-03-01
3,-,51.0,48.6,-,51.8,57.0,50.9,54.6,50.0,49.3,...,53.5,51.7,51.6,47.6,51.8,52.5,49.7,48.7,53.7,2010-04-01
4,-,57.3,50.5,-,53.5,55.7,48.1,55.4,53.7,59.4,...,59.7,57.2,54.3,49.5,54.9,53.7,54.9,55.7,57.2,2010-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
131,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
132,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
133,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [10]:
df_dados = df_dados.dropna()

In [11]:
df_dados.columns[2]

'Móveis'

In [12]:
df_dados = df_dados.rename(columns={'Manutenção, reparação e instalação de máquinas e equipamentos.': 'manutencao reparacao e instalacao de maquinas e equipamentos'
                                ,'Produtos diversos': 'produtos diversos'
                                ,'Móveis': 'moveis'
                                ,'Outros equipamentos de transporte': 'outros equipamentos de transporte'
                                ,'Veículos automotores, reboques e carrocerias ': 'veiculos automotores reboques e carrocerias'
                                ,'Máquinas e equipamentos': 'maquinas e equipamentos'
                                ,'Máquinas, aparelhos e materiais elétricos ': 'maquinas aparelhos e materiais eletricos'
                                ,'Equipamentos de informática, produtos eletrônicos e outros ': 'equipamentos de informatica produtos eletronicos e outros'
                                ,'Produtos de metal (exceto máquinas e equipamentos)  ': 'produtos de metal'
                                ,'Metalurgia': 'metalurgia'
                                ,'Produtos de minerais não metálicos': 'produtos de minerais nao metalicos'
                                ,'Produtos de material plástico': 'produtos de material plastico'
                                ,'Produtos de borracha': 'produtos de borracha'
                                ,'Produtos farmoquímicos e farmacêuticos ': 'produtos farmoquimicos e farmaceuticos'
                                ,'Sabões, detergentes, produtos de limpeza, cosméticos, produtos de perfumaria e de higiene pessoal (HPPC)': 'hppc'
                                ,'Químicos (exceto HPPC)': 'quimicos'
                                ,'Biocombustíveis': 'biocombustiveis'
                                ,'Coque e derivados do petróleo': 'coque e derivados do petroleo'
                                ,'Impressão e reprodução de gravações': 'impressao e reproducao de gravacoes'
                                ,'Celulose, papel e produtos de papel': 'celulose papel e produtos de papel'
                                ,'Produtos de madeira': 'produtos de madeira'
                                ,'Calçados e suas partes': 'calcados e suas partes'
                                ,'Couros e artefatos de couro': 'couros e artefatos de couro'
                                ,'Confecção de artigos do vestuário e acessórios ': 'confeccao de artigos do vestuario e acessorios'
                                ,'Produtos têxteis': 'produtos texteis'
                                ,'Produtos do fumo': 'produtos do fumo'
                                ,'Bebidas': 'bebidas'
                                ,'Produtos alimentícios': 'produtos alimenticios'
                                ,'IND. DE TRANSFORMAÇÃO': 'industria de transformacao'
                                ,'Atividades de apoio à extração de minerais': 'atividades de apoio a extracao de minerais'
                                ,'Extração de minerais não metálicos': 'extracao de minerais nao metalicos'
                                ,'Extração de minerais metálicos': 'extracao de minerais metalicos'
                                ,'Extração de carvão, petróleo e gás natural': 'extracao de carvao petroleo e gas natural'
                                ,'IND. EXTRATIVA': 'industria extrativa'
                                ,'GRANDE': 'industria grande'
                                ,'MÉDIA': 'industria media'
                                ,'PEQUENA ': 'industria pequena'
                                ,'CENTRO-OESTE': 'regiao centro oeste'
                                ,'SUL': 'regiao sul'
                                ,'SUDESTE': 'regiao sudeste'
                                ,'NORDESTE': 'regiao nordeste'
                                ,'NORTE': 'regiao norte'
                                ,'Data': 'data'
                            })
df_dados.head()

,manutencao reparacao e instalacao de maquinas e equipamentos,produtos diversos,moveis,outros equipamentos de transporte,veiculos automotores reboques e carrocerias,maquinas e equipamentos,maquinas aparelhos e materiais eletricos,equipamentos de informatica produtos eletronicos e outros,produtos de metal,metalurgia,...,industria extrativa,industria grande,industria media,industria pequena,regiao centro oeste,regiao sul,regiao sudeste,regiao nordeste,regiao norte,data
0,-,50.0,42.7,-,53.6,51.6,50.0,50.6,50.0,51.1,...,48.5,51.3,46.5,46.5,48.6,48.9,48.1,49.0,53.7,2010-01-01
1,-,56.9,47.1,-,52.7,57.6,50.0,58.3,48.6,57.3,...,48.4,52.3,50.0,48.0,52.0,55.3,49.4,46.1,49.3,2010-02-01
2,-,62.5,63.3,-,67.5,64.8,65.2,65.7,61.0,65.7,...,58.1,65.2,62.8,56.3,61.5,64.0,64.3,59.4,60.9,2010-03-01
3,-,51.0,48.6,-,51.8,57.0,50.9,54.6,50.0,49.3,...,53.5,51.7,51.6,47.6,51.8,52.5,49.7,48.7,53.7,2010-04-01
4,-,57.3,50.5,-,53.5,55.7,48.1,55.4,53.7,59.4,...,59.7,57.2,54.3,49.5,54.9,53.7,54.9,55.7,57.2,2010-05-01


In [13]:
#Tratando dados

df_dados = df_dados.replace('-','0')
df_dados = df_dados.replace('.',',')
df_dados

,manutencao reparacao e instalacao de maquinas e equipamentos,produtos diversos,moveis,outros equipamentos de transporte,veiculos automotores reboques e carrocerias,maquinas e equipamentos,maquinas aparelhos e materiais eletricos,equipamentos de informatica produtos eletronicos e outros,produtos de metal,metalurgia,...,industria extrativa,industria grande,industria media,industria pequena,regiao centro oeste,regiao sul,regiao sudeste,regiao nordeste,regiao norte,data
0,0,50.0,42.7,0,53.6,51.6,50.0,50.6,50.0,51.1,...,48.5,51.3,46.5,46.5,48.6,48.9,48.1,49.0,53.7,2010-01-01
1,0,56.9,47.1,0,52.7,57.6,50.0,58.3,48.6,57.3,...,48.4,52.3,50.0,48.0,52.0,55.3,49.4,46.1,49.3,2010-02-01
2,0,62.5,63.3,0,67.5,64.8,65.2,65.7,61.0,65.7,...,58.1,65.2,62.8,56.3,61.5,64.0,64.3,59.4,60.9,2010-03-01
3,0,51.0,48.6,0,51.8,57.0,50.9,54.6,50.0,49.3,...,53.5,51.7,51.6,47.6,51.8,52.5,49.7,48.7,53.7,2010-04-01
4,0,57.3,50.5,0,53.5,55.7,48.1,55.4,53.7,59.4,...,59.7,57.2,54.3,49.5,54.9,53.7,54.9,55.7,57.2,2010-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,46.4,43.8,42.9,45.8,48.3,47.8,46.0,44.7,47.1,50.8,...,46.3,50.9,45.2,43.2,48.2,51.7,46.2,46.0,43.2,2020-02-01
122,34.6,30.3,14.2,28.8,30.2,31.0,36.1,31.1,26.7,34.4,...,45.6,35.0,35.1,28.0,34.4,30.4,32.4,30.3,35.2,2020-03-01
123,28.7,17.6,9.7,23.4,16.8,26.1,22.0,22.9,22.3,24.2,...,43.0,26.6,26.9,24.0,39.0,22.8,25.7,23.2,27.7,2020-04-01
124,44.3,38.7,40.2,42.6,49.2,45.5,36.1,34.8,40.0,41.5,...,48.4,47.2,42.4,35.6,50.0,45.3,41.9,41.2,39.9,2020-05-01


In [14]:
#SALVANDO DATAFRAME EM CSV

save_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\csv\dados_industria'+ '_' + sheets + '.csv'

df_dados.to_csv(save_filepath, decimal = ",", sep = ";")